vamos a empezar de nuevo con lo del sop para ver si sale

In [ ]:
import numpy as np
import random 
import csv
from operator import itemgetter

In [ ]:
#funcion para obtener el costo

def obtenerCosto (solucion, costos, reglas):
  #obtener el total de nodos -1 (no es necesario revisar el ultimo nodo)
  nodos = len(solucion) -1

  #se obtine el costo máximode la tabla
  costoMax = max (max(fila) for fila in a)
  costoTotal = 0

  #se suman los costos C de la solución
  for i in range(nodos):
    costoTotal += costos[solucion[i]][solucion[i+1]]
  n =  presedencia(solucion, reglas)
  costoTotal += n* costoMax  

In [ ]:
def presedencia(solucion, rules):
  #tiene que variar estos arreglos dependiendo de la cantidad de reglas
  auxB = np.zeros(len(rules))
  cont = 0

  for i in range (len(solucion)):
    for j in range(len(rules)):
      if rules[j][0] == solucion[i]:
        auxB[j]+=1
      if auxB[j]== 1:
        if rules[j][1] == solucion[i]:
          auxB[j]+=1
  for i in range(len(auxB)):
    if auxB[i] <2:
      cont +=1
  return cont

In [ ]:
def solucionInicial(numNodos,reglas):
    solucionTemp = []
    solucion = []
    #generacion de numeros aleatorios con su indice para obtener la primera solucion
    for i in range(numNodos-2):
        solucionTemp.append([i+1,random.random()])
    #Se reordenan para poder decidir en que orden visitar los nodos
    solucionTemp.sort(key=itemgetter(1))
    # El primero nodo debe ser siempre el cero
    solucion.append(0)
    #agregar los demas nodos
    for i in list(solucionTemp):
        solucion.append(i[0])
    #El ultimo nodo debe ser numNodos-1
    solucion.append(numNodos-1)
    solucion = corregirPrecedencia(solucion[:],reglas,numNodos)
    return solucion

In [ ]:
def solucionVecino(solucion,reglas):
    #generar las posiciones a cambiar
    pos1 = random.randint(1,len(solucion)-2)
    pos2 = random.randint(1,len(solucion)-2)
    while (pos1 == pos2):
        pos2 = random.randint(1,len(solucion)-2)
    #print(pos1, ',', pos2)
    #Relizamos el cambio de acuerdo a los indices 
    solucion[pos1],solucion[pos2] = solucion[pos2],solucion[pos1]
    solucion = corregirPrecedencia(solucion[:],reglas,len(solucion))
    return solucion

In [ ]:
def corregirPrecedencia(solucion, reglas, numNodos):
    # Se recorre la lista de reglas
    for regla in reglas:
        nodoA = regla[0]
        nodoB = regla[1]

        # Se busca la posición de los nodos en la solución actual
        posA = solucion.index(nodoA)
        posB = solucion.index(nodoB)

        # Si la posición de B es menor que la de A, se intercambian los nodos
        if posB < posA:
            solucion[posA], solucion[posB] = solucion[posB], solucion[posA]

    # Se verifica que la solución corregida contenga todos los nodos
    assert set(solucion) == set(range(numNodos))

    return solucion


In [ ]:
def readFile(size, name):
    m = np.zeros((0), dtype=int)
    m_prec = np.zeros((0), dtype=int)
    
    with open(name, newline='') as File:  
        reader = csv.reader(File, delimiter=',', quotechar=',',quoting=csv.QUOTE_MINIMAL)
        i=0
        for row in File:
            
            row = row.rstrip()
            separador = ","
            row = row.split(",")
            row = list(map(int, row))
            m_np = np.array(row)
            
            if i < size :
                m = np.append(m, m_np, axis=0)
                #print(m_np)
            else:
                m_prec = np.append(m_prec, m_np, axis=0)
            i+=1
    
    m = np.array(m).reshape(size,size)
    m_prec = np.array(m_prec).reshape(int((len(m_prec)/2)),2)
    return m, m_prec

In [ ]:
name = '100.txt'
nodos = 100
m, m_prec = readFile(nodos, name)

print(f'{m}')


[[19  8 13 ...  2  2  4]
 [ 2  6  3 ... 10  8  6]
 [ 3 14 10 ... 20 13 11]
 ...
 [13 10 14 ... 18 19 20]
 [ 3 13 16 ...  0  0 18]
 [ 8  1  0 ...  4  0 12]]


In [ ]:
print(m_prec)

[[ 1  3]
 [ 1  4]
 [ 1 13]
 [ 8  7]
 [15 22]
 [48 49]]


ES UNA SEPARACION PARA PROBAR LO DER RECOCIDO JEJEJEO

In [ ]:
import random
import math

# Función para calcular la energía de una solución
def calcularEnergia(solucion, costos):
    energia = 0
    for i in range(len(solucion) - 1):
        energia += costos[solucion[i]][solucion[i + 1]]
    return energia

# Función de recocido simulado para el SOP
def recocidoSimulado(numNodos, costos, reglas, T=10.0, T_min=0.0001, enfriamiento=0.99):
    # Generar la solución inicial y corregir las precedencias
    solucion_actual = solucionInicial(numNodos, reglas)
    solucion_actual = corregirPrecedencia(solucion_actual, reglas, numNodos)
    
    # Calcular la energía de la solución inicial
    energia_actual = calcularEnergia(solucion_actual, costos)

    # Inicializar la mejor solución encontrada
    mejor_solucion = solucion_actual.copy()
    mejor_energia = energia_actual

    # Iniciar el recocido simulado
    while T > T_min:
        # Generar una solución vecina
        vecino = solucion_actual.copy()
        idx = random.sample(range(1, numNodos - 1), 2)
        vecino[idx[0]], vecino[idx[1]] = vecino[idx[1]], vecino[idx[0]]
        vecino = corregirPrecedencia(vecino, reglas, numNodos)

        # Calcular la energía del vecino
        energia_vecino = calcularEnergia(vecino, costos)

        # Si la energía del vecino es mejor, se actualiza la solución actual
        if energia_vecino < energia_actual:
            solucion_actual = vecino
            energia_actual = energia_vecino

            # Si la solución actual es mejor que la mejor solución, se actualiza la mejor solución
            if energia_actual < mejor_energia:
                mejor_solucion = solucion_actual.copy()
                mejor_energia = energia_actual

        # Si la energía del vecino es peor, se acepta con una probabilidad determinada
        else:
            delta_energia = energia_vecino - energia_actual
            probabilidad_aceptar = math.exp(-delta_energia / T)
            if random.random() < probabilidad_aceptar:
                solucion_actual = vecino

        # Enfriar la temperatura
        T *= enfriamiento

    # Devolver la mejor solución encontrada
    return mejor_solucion, mejor_energia


In [ ]:
best_solution, best_combo = recocidoSimulado(numNodos= 100, costos = m, reglas = m_prec)

print(f'best solution {best_combo} with best combination {best_solution}')

best solution 316 with best combination [0, 81, 18, 60, 15, 23, 22, 26, 44, 41, 21, 24, 83, 92, 95, 52, 2, 74, 25, 94, 38, 1, 75, 36, 61, 8, 91, 42, 28, 71, 32, 34, 62, 59, 13, 66, 67, 7, 80, 50, 4, 43, 63, 33, 88, 76, 93, 45, 57, 6, 27, 87, 84, 78, 29, 3, 51, 55, 70, 37, 89, 47, 56, 72, 16, 77, 48, 54, 14, 96, 12, 53, 58, 86, 11, 82, 31, 5, 20, 73, 90, 39, 79, 46, 9, 65, 64, 10, 17, 69, 40, 68, 30, 19, 98, 49, 85, 97, 35, 99]
